In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
# tf.GradientTape : 자동 미분(주어진 입력 변수에 대한 연산의 Gradient를 계산하는 것)

x = tf.Variable(3.0)

with tf.GradientTape() as tape :
  y = x**2

In [3]:
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0

In [5]:
w = tf.Variable(tf.random.normal((3, 2)), name = 'w')
b = tf.Variable(tf.zeros(2, dtype = tf.float32), name = 'b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)


두 변수 모두에 대해 loss의 그래디언트를 가져오려면, 두 변수를 gradient 메서드에 소스로 전달할 수 있습니다. 테이프는 소스가 전달되는 방식에 대해 융통성이 있으며 목록 또는 사전의 중첩된 조합을 허용하고 같은 방식으로 구조화된 그래디언트를 반환합니다

In [6]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b]) # 각 소스에 대한 그래디언트는 소스의 형상을 갖는다

print(w.shape)
print(dl_dw.shape)

(3, 2)
(3, 2)


In [7]:
# 변수에 dictionary를 전달하는 그래디언트 계산
my_vars = {'w':w, 'b':b}

grad = tape.gradient(loss, my_vars)
grad['b']

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 4.1267757 , -0.20443402], dtype=float32)>

In [9]:
# 모델에 대한 그래디언트

layer = tf.keras.layers.Dense(2, activation = 'relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

# Calculate gradients with respect to every trainable variable
grad = tape.gradient(loss, layer.trainable_variables)

In [10]:
for var, g in zip(layer.trainable_variables, grad):
  print(f'{var.name}, shape:{g.shape}')

dense/kernel:0, shape:(3, 2)
dense/bias:0, shape:(2,)


### tape의 감시 대상 제어하기

기본 동작은 훈련 가능한 tf.Variable에 액세스한 후 모든 연산을 기록하는 것. 그 이유는 다음과 같다.

* 테이프는 역방향 패스의 그래디언트를 계산하기 위해 정방향 패스에 기록할 연산을 알아야 한다.
* 테이프는 중간 출력에 대한 참조를 보유하므로 불필요한 연산을 기록하지 않는다.
* 가장 일반적인 사용 사례는 모든 모델의 훈련 가능한 변수에 대해 손실의 그래디언트를 계산하는 것이다.
  
예를 들어, 다음은 tf.Tensor가 기본적으로 "감시"되지 않고 tf.Variable을 훈련할 수 없기 때문에 그래디언트를 계산하지 못한다.

In [11]:
# A trainable variable
x0 = tf.Variable(3.0, name = 'x0')
# Not trainable
x1 = tf.Variable(3.0, name = 'x1', trainable=False)
# Not a Variable: A variable + tensor returns a tensor.
x2 = tf.Variable(2.0, name = 'x2') + 1.0
# Not a Variable
x3 = tf.constant(3.0, name = 'x3')

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:
  print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


In [12]:
# GradientTape.watched_variables : tape에서 감시중인 변수를 나열

[var.name for var in tape.watched_variables()]

['x0:0']

In [13]:
# tf.GradientTape는 사용자가 감시 대상 또는 감시 예외 대상을 제어할 수 있는 후크를 제공
# tf.Tensor에 대한 그래디언트를 기록하려면 GradientTape.watch(x)를 호출

x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x**2

# dy = 2x * dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

6.0


반대로, 모든 tf.Variables을 감시하는 기본 동작을 비활성화하려면, 그래디언트 테이프를 만들 때 watch_accessed_variables=False를 설정한다. 이 계산은 두 가지 변수를 사용하지만, 변수 중 하나의 그래디언트만 연결한다.

In [14]:
x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

x0에서 GradientTape.watch가 호출되지 않았으므로 이에 대한 그래디언트가 계산되지 않는다.


In [15]:
# dys/dx1 = exp(x1) / (1 + exp(x1)) = sigmoid(x1)
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0', grad['x0'])
print('dy/dx1', grad['x1'].numpy())

dy/dx0 None
dy/dx1 0.9999546
